## How to use

In [1]:
from repoLibrarian import *

In [7]:
list(knownRepos())

['bptlab/scylla.git', 'bptlab/fcm2cpn.git']

In [3]:
print(hasRepo('bptlab', 'scylla'))
print(hasRepo('bptlab', 'fcm2cpn'))

False
True


In [3]:
user = 'bptlab'
project = 'scylla'
downloadRepo(user, project)

Repo "bptlab/scylla" already exists locally


In [4]:
deleteRepo('bptlab', 'fcm2cpn')

Repo "bptlab/fcm2cpn" does not exist locally


In [6]:
getRepo('bptlab', 'fcm2cpn')

Cloned repo "bptlab/fcm2cpn"


<git.repo.base.Repo '/mnt/brick/crm20/repos/bptlab/fcm2cpn.git'>

# Java User projects

In [15]:
from repoLibrarian import *
from utils import *
import utils

In [14]:
%load_ext autoreload
%autoreload

In [16]:
runQuery('''
        CREATE MATERIALIZED VIEW crm20.lb_polyglots AS
            (SELECT lb_java_users.author_id, lb_java_users.loc AS javaLoC,  lb_java_users.first_written as javaStart, ARRAY_AGG(lb_languages.language) AS secondary_languages, ARRAY_AGG(lb_languages.loc) AS secLoC, ARRAY_AGG(lb_languages.first_written) as secStart
            FROM crm20.lb_java_users, crm20.lb_languages
            WHERE lb_java_users.author_id = lb_languages.author_id 
            AND (lb_languages.language = 'py' OR lb_languages.language = 'js') AND lb_languages.loc > 1000
            GROUP BY lb_java_users.author_id, lb_java_users.loc, lb_java_users.first_written
            HAVING lb_java_users.loc / 2 > ALL(ARRAY_AGG(lb_languages.loc))
            AND lb_java_users.first_written < ALL(ARRAY_AGG(lb_languages.first_written)));
        SELECT * FROM crm20.lb_polyglots
''')

Time used: 0.1988239288330078


,author_id,javaloc,javastart,secondary_languages,secloc,secstart
0,69602,149818,2012-07-06 20:21:47,[js],[14102],[2012-11-16 18:22:57]
1,1275950,79795,2014-01-21 13:25:31,[py],[3020],[2014-03-29 16:39:49]
2,1979246,34247,2013-06-06 05:28:21,[js],[7991],[2013-06-10 07:17:46]
3,2200247,18317,2014-09-10 21:07:58,[py],[2234],[2014-10-09 18:11:46]
4,2725277,31015,2014-05-28 08:19:28,[js],[9449],[2014-06-07 08:30:32]
5,3242981,20263,2014-01-30 16:13:29,[js],[2296],[2014-11-08 16:28:25]
6,4908425,13493,2014-08-15 04:49:30,[js],[5306],[2015-09-27 06:27:28]
7,5080586,42824,2014-08-25 10:38:05,[js],[18614],[2014-09-16 14:15:56]
8,9297920,17813,2015-09-22 00:45:02,[py],[1397],[2018-04-23 12:41:53]
9,10328444,48735,2016-03-09 21:08:22,[py],[4419],[2016-09-08 14:13:20]


In [18]:
polyglotProjects = runQuery('''
    SELECT DISTINCT repo_id
    FROM lb_polyglots, ght.project_members 
    WHERE author_id = user_id
''')

Time used: 0.3478097915649414


,repo_id
0,21398
1,128825
2,646761
3,1408482
4,1679896
...,...
110,112264323
111,113106393
112,116556763
113,119748529


In [40]:
basicJavaRepos = runQuery('''
    SELECT DISTINCT repo_id, url, name
    FROM lb_polyglots, ght.project_members, ght.projects
    WHERE author_id = user_id
    AND repo_id = id
    AND language = 'Java'
''')
display(basicJavaRepos)

Time used: 0.4696311950683594


,repo_id,url,name
0,21398,https://api.github.com/repos/bakis2011/PocketVDC,PocketVDC
1,128825,https://api.github.com/repos/scssquatch/python...,python_koans
2,646761,https://api.github.com/repos/bakis2011/hello_s...,hello_sinatra
3,1408482,https://api.github.com/repos/bakis2011/Lab7,Lab7
4,1679896,https://api.github.com/repos/scssquatch/r-webdev,r-webdev
...,...,...,...
110,112264323,https://api.github.com/repos/Chaitya62/Excusem...,ExcusemeWTF
111,113106393,https://api.github.com/repos/alstonlo/Call,Call
112,116556763,https://api.github.com/repos/YashArora0606/Sig...,SignInSystem
113,119748529,https://api.github.com/repos/alstonlo/Chat-Sof...,Chat-Software-Assignment


In [38]:
for url in basicJavaRepos['url']:
    split = url.split('/')
    user = split[-2]
    project = split[-1]
    downloadRepo(user, project)

Could not download repo "bakis2011/PocketVDC": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/bakis2011/PocketVDC.git /mnt/brick/crm20/repos/bakis2011/PocketVDC.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/bakis2011/PocketVDC.git'...
fatal: could not read Username for 'https://github.com': No such device or address
'
Could not download repo "bakis2011/Lab7": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/bakis2011/Lab7.git /mnt/brick/crm20/repos/bakis2011/Lab7.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/bakis2011/Lab7.git'...
fatal: could not read Username for 'https://github.com': No such device or address
'
Could not download repo "bakis2011/Lab8": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/bakis2011/Lab8.git /mnt/brick/crm20/repos/bakis2011/Lab8.git
  stderr: 'Cloning into bare repository '/mnt/brick/cr

In [41]:
basicAllRepos = runQuery('''
    SELECT DISTINCT repo_id, url, name
    FROM lb_polyglots, ght.project_members, ght.projects
    WHERE author_id = user_id
    AND repo_id = id
''')
display(basicAllRepos)

Time used: 0.4717867374420166


,repo_id,url,name
0,21398,https://api.github.com/repos/bakis2011/PocketVDC,PocketVDC
1,128825,https://api.github.com/repos/scssquatch/python...,python_koans
2,646761,https://api.github.com/repos/bakis2011/hello_s...,hello_sinatra
3,1408482,https://api.github.com/repos/bakis2011/Lab7,Lab7
4,1679896,https://api.github.com/repos/scssquatch/r-webdev,r-webdev
...,...,...,...
110,112264323,https://api.github.com/repos/Chaitya62/Excusem...,ExcusemeWTF
111,113106393,https://api.github.com/repos/alstonlo/Call,Call
112,116556763,https://api.github.com/repos/YashArora0606/Sig...,SignInSystem
113,119748529,https://api.github.com/repos/alstonlo/Chat-Sof...,Chat-Software-Assignment


In [47]:
for url in basicAllRepos['url']:
    split = url.split('/')
    user = split[-2]
    project = split[-1]
    repo = downloadRepo(user, project)

Could not download repo "bakis2011/PocketVDC": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/bakis2011/PocketVDC.git /mnt/brick/crm20/repos/bakis2011/PocketVDC.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/bakis2011/PocketVDC.git'...
fatal: could not read Username for 'https://github.com': No such device or address
'
Could not download repo "scssquatch/python_koans": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/scssquatch/python_koans.git /mnt/brick/crm20/repos/scssquatch/python_koans.git
  stderr: 'Cloning into bare repository '/mnt/brick/crm20/repos/scssquatch/python_koans.git'...
fatal: could not read Username for 'https://github.com': No such device or address
'
Could not download repo "bakis2011/hello_sinatra": Cmd('git') failed due to: exit code(128)
  cmdline: git clone --bare -v https://github.com/bakis2011/hello_sinatra.git /mnt/brick/crm20/repos/bakis2011/hello_sin